<h1>  Build agentic AI application using Langgraph </h1>
<p> Built by Jakob Lindstrøm, aka DataJakob@Github,  for Lumos  SDC. </p>
<p> In this notebook  we will create a agentic chatbot that is especially good at Q&A for finance and portfolio generation </p> 

<h3> 1 Import libraries </h3>

In [1]:
# Enviroment 
import os

# "Lang" packages
import langchain
import langgraph
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field
from langgraph.checkpoint.memory import MemorySaver
# Other
from demo.analyze.data import PortofolioCloud
from demo.analyze.optimizer import Optimized

# .env variables
# my_api_key = os.getenv("OPENAI_API_KEY")

<h2> 2 State and classes </h2>

In [225]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    mission: str
    company_info: str
    stocks: Annotated[list, add_messages]
    optimal_portfolio: str = Field(description="""A sentence describing the optimal stock posistion in a portfolio
                                                based on the Sharpe ratio""")


class Mission(BaseModel):
    mission: str = Field(description="""
                         Answer with one word based on the user query:
                         - "append", if the user wants to append  the stock into the portfolio.
                         - "more_info"  if the user wants more info about a stock or something else.
                         - "analyze"  if  the user wants to analyze the current portfolio,
                         - "other", if nothing of the above is specified.
                        """)
    
class Filter(BaseModel):
    ticker: str = Field(description="""
                        Identify the financial ticker of the mentioned company. 
                        """)

<h2> 3 Model and Nodes </h2>

In [130]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    api_key=my_api_key
)


def chatbot(system_message, user_message):
    system_input = "You are chatbot specializing in financial advisory. " + str(system_message)
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_input),
    ])
    info_model = prompt | llm 
    response =  info_model.invoke({"user":user_message})
    return response.content

In [ ]:
def navigator(state: State):
    nav_model =  llm.with_structured_output(Mission)
    response = nav_model.invoke(state["messages"]).mission
    return {"mission": response}



def info(state: State):
    response = chatbot("You shall give a short description of what a company does based on the user input.",
                       state["messages"]
                       )
    return {"company_info": response.content}


    
def stock_appender(state: State):
    nav_model =  llm.with_structured_output(Filter)
    response = nav_model.invoke(state["messages"]).ticker+".OL"
    print(state["stocks"])
    return {"stocks": response}



def portfolio_analyzer(state: State):
    stocks = [x.content for x in state["stocks"]]
    print(stocks)
    alfa = PortofolioCloud(stocks)
    alfa.final_df()
    bravo = Optimized(alfa.stocks, alfa.df,
                    alfa.tot_cov_mat,alfa.neg_cov_mat,
                    alfa.returns, alfa.mean, alfa.std)
    bravo.PortofolioOptimizer()
    positions =  bravo.optimized_portofolios
    myList = ""
    for i in range(0, len(stocks),1):
        myList += str(stocks[i]+ " at " +str(round(positions[2][0][i]*100,1))+"%, ")
    print(myList)

    return {"optimal_portfolio": myList}



def final_response(state: State):
    if state["mission"] == "other":
        response = chatbot("""The user has given you a unrelated query, make the user give queries about appending stocks to a porfolio,
                           get information about a company or analyze a portfolio """,
                           state["messages"]
                           )
        return {"messages": response}

    elif state["mission"] == "analyze":
        response = chatbot("""Use the input from the user to inform the user about the stock posistion that would generate
                           the best portfolio based on the Sharpe ratio.""",
                            state["optimal_portfolio"]
                            )
        return {"messages": response}
    
    elif state["mission"] == "append":
        response = chatbot("""You have just added a stock to a portfolio, ask the user if a new stock should be added.
                            Or if the user wants to get info about a company or  analyze the portfolio""",
                            state["messages"]
                            )
        return {"messages": response}
        
    else:
        response = chatbot(""" Give key pieces of information about a company that the user have requested.""",
                           state["messages"]
                           )
        return {"messages": response}

<h2> 4 Edges</h2>

In [232]:
def decision(state: State):
    if state["mission"] == "append":
        return "add_stock"
    elif state["mission"] == "analyze":
        return  "analyze_portfolio"
    elif state["mission"] == "info":
        return "more_info"
    else:
        return "other"

<h2> 5 Workflow Compilation </h2>

In [189]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [227]:
# Initialize the graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("nav", navigator)
workflow.add_node("info", info)
workflow.add_node("append", stock_appender)
workflow.add_node("analyze", portfolio_analyzer)
workflow.add_node("response", final_response)

# Add edges
workflow.add_edge(START, "nav")     # Non-optional move
workflow.add_conditional_edges("nav",   # Starting node
                               decision,    # Decider for which node to go to
                               # Options
                               {"more_info":"info",
                                "add_stock":"append",
                                "analyze_portfolio":"analyze",
                                "other":"response"}

)
workflow.add_edge("info", "response")
workflow.add_edge("append", "response")
workflow.add_edge("analyze", "response")
workflow.add_edge("response", END)

# Compile and save the graph
# workflow.compile()
mygraph = workflow.compile(checkpointer=memory)

<h2> 6 Start chatting :) </h2>

In [228]:
def stream_graph_updates(user_input: str):
    for event in mygraph.stream({"messages": [("user", user_input)]},
                                {"configurable": {"thread_id": "0a"}}):
        for value in event:
            print("Assistant:", value)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "----Something wong here------"    
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Assistant: nav
['EQNR.OL', 'MOWI.OL']



c:\Users\Jakob\OneDrive\Dokumenter\master\Lumos\langgraph\.venvLSDC\Lib\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


EQNR.OL at 48.1%, MOWI.OL at 51.9%, 
Assistant: analyze
Assistant: response


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Assistant: nav
['EQNR.OL', 'MOWI.OL']



c:\Users\Jakob\OneDrive\Dokumenter\master\Lumos\langgraph\.venvLSDC\Lib\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


EQNR.OL at 48.1%, MOWI.OL at 51.9%, 
Assistant: analyze
Assistant: response
Goodbye!
